**Question 1**

In [2]:
import math
import random
from collections import defaultdict
import heapq

# Sample Data
drones = {
    "Drone1": {"speed": 50, "battery_capacity": 100},
    "Drone2": {"speed": 40, "battery_capacity": 120},
    "Drone3": {"speed": 60, "battery_capacity": 90},
}

packages = {
    "P1": {"location": (10, 20), "deadline": 10, "priority": 1},
    "P2": {"location": (15, 25), "deadline": 15, "priority": 2},
    "P3": {"location": (30, 5), "deadline": 20, "priority": 1},
    "P4": {"location": (5, 35), "deadline": 12, "priority": 3},
    "P5": {"location": (25, 10), "deadline": 18, "priority": 2},
    "P6": {"location": (18, 22), "deadline": 25, "priority": 1},
}

recharging_stations = [(0, 0), (50, 50)]
origin = (0, 0)

no_fly_zones = [
    {"location": (20, 20), "radius": 5, "peak_hours": (8, 18)}
]

# Time is simulated in hours from 0 onwards
def euclidean_distance(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

# Constraints
def is_battery_feasible(route, drone):
    total = 0
    battery = drones[drone]["battery_capacity"]
    current = origin
    for point in route:
        if point == "RECHARGE":
            total = 0
            current = origin
        else:
            dist = euclidean_distance(current, packages[point]["location"])
            total += dist
            if total > battery:
                return False
            current = packages[point]["location"]
    return True

def violates_deadlines(route, drone):
    current = origin
    speed = drones[drone]["speed"]
    time = 0
    for point in route:
        if point == "RECHARGE":
            time += 1  # 1 hour to recharge
            current = origin
        else:
            dist = euclidean_distance(current, packages[point]["location"])
            time += dist / speed
            if time > packages[point]["deadline"]:
                return True
            current = packages[point]["location"]
    return False

def violates_priority_assignment(assignments):
    for pkg, drone in assignments.items():
        if packages[pkg]["priority"] == 1 and drones[drone]["speed"] < 50:
            return True
    return False

# AC-3 Simplified
def ac3(domains):
    return True

# MRV
def select_unassigned_variable(assignment, domains):
    unassigned = [v for v in domains if v not in assignment]
    return min(unassigned, key=lambda var: len(domains[var]))

# LCV
def order_domain_values(var, assignment, domains):
    drone_costs = []
    for drone in domains[var]:
        route = [pkg for pkg, d in assignment.items() if d == drone] + [var]
        drone_route = generate_route_with_recharge(route, drone)
        cost = compute_cost(drone_route, drone)
        heapq.heappush(drone_costs, (cost, drone))
    return [drone for cost, drone in sorted(drone_costs)]

def generate_route_with_recharge(route, drone):
    total_dist = 0
    current = origin
    new_route = []
    battery = drones[drone]["battery_capacity"]

    for p in route:
        dist = euclidean_distance(current, packages[p]["location"])
        if total_dist + dist > battery:
            new_route.append("RECHARGE")
            total_dist = 0
            current = origin
        new_route.append(p)
        total_dist += dist
        current = packages[p]["location"]
    return new_route

def compute_cost(route, drone):
    speed = drones[drone]["speed"]
    current = origin
    time = 0
    battery_used = 0
    delay_penalty = 0

    for point in route:
        if point == "RECHARGE":
            time += 1
            battery_used = 0
            current = origin
        else:
            dist = euclidean_distance(current, packages[point]["location"])
            time += dist / speed
            battery_used += dist
            deadline = packages[point]["deadline"]
            delay_penalty += max(0, time - deadline)
            current = packages[point]["location"]
    return battery_used + 10 * delay_penalty

def compute_delivery_time(route, drone):
    speed = drones[drone]["speed"]
    current = origin
    time = 0
    delivery_times = {}

    for point in route:
        if point == "RECHARGE":
            time += 1
            current = origin
        else:
            dist = euclidean_distance(current, packages[point]["location"])
            time += dist / speed
            delivery_times[point] = round(time, 2)
            current = packages[point]["location"]
    return delivery_times

# Backtracking Search
def backtrack(assignment, domains):
    if len(assignment) == len(packages):
        if violates_priority_assignment(assignment):
            return None
        return assignment

    var = select_unassigned_variable(assignment, domains)
    for value in order_domain_values(var, assignment, domains):
        local_assignment = assignment.copy()
        local_assignment[var] = value
        routes = defaultdict(list)
        for pkg, dr in local_assignment.items():
            routes[dr].append(pkg)

        valid = True
        for dr, route in routes.items():
            full_route = generate_route_with_recharge(route, dr)
            if not is_battery_feasible(full_route, dr) or violates_deadlines(full_route, dr):
                valid = False
                break

        if valid:
            if ac3(domains):
                result = backtrack(local_assignment, domains)
                if result:
                    return result
    return None

# Domains
domains = {pkg: list(drones.keys()) for pkg in packages}
final_assignment = backtrack({}, domains)

# Final Output
drone_to_packages = defaultdict(list)
for pkg, dr in final_assignment.items():
    drone_to_packages[dr].append(pkg)

results = {}
total_cost = 0

for drone, pkg_list in drone_to_packages.items():
    full_route = generate_route_with_recharge(pkg_list, drone)
    delivery_times = compute_delivery_time(full_route, drone)
    cost = compute_cost(full_route, drone)
    total_cost += cost
    results[drone] = {
        "route": full_route,
        "delivery_times": delivery_times,
        "cost": round(cost, 2)
    }

results["Total_Operational_Cost"] = round(total_cost, 2)
results


{'Drone1': {'route': ['P1', 'P4'],
  'delivery_times': {'P1': 0.45, 'P4': 0.76},
  'cost': 38.17},
 'Drone2': {'route': ['P2'], 'delivery_times': {'P2': 0.73}, 'cost': 29.15},
 'Drone3': {'route': ['P3', 'P5', 'P6'],
  'delivery_times': {'P3': 0.51, 'P5': 0.62, 'P6': 0.86},
  'cost': 51.38},
 'Total_Operational_Cost': 118.7}

**Question 2**

In [3]:
# Connect Four Game Logic

ROWS = 6
COLUMNS = 7
PLAYER_PIECE = 1
AI_PIECE = 2
EMPTY = 0

def create_board():
    return [[0 for _ in range(COLUMNS)] for _ in range(ROWS)]

def print_board(board):
    print("\n".join([" ".join(str(cell) for cell in row) for row in reversed(board)]))
    print("1 2 3 4 5 6 7\n")

def is_valid_location(board, col):
    return board[ROWS-1][col] == 0

def get_next_open_row(board, col):
    for r in range(ROWS):
        if board[r][col] == 0:
            return r

def drop_piece(board, row, col, piece):
    board[row][col] = piece

def winning_move(board, piece):
    # Horizontal check
    for c in range(COLUMNS - 3):
        for r in range(ROWS):
            if all(board[r][c+i] == piece for i in range(4)):
                return True
    # Vertical check
    for c in range(COLUMNS):
        for r in range(ROWS - 3):
            if all(board[r+i][c] == piece for i in range(4)):
                return True
    # Positive diagonal
    for c in range(COLUMNS - 3):
        for r in range(ROWS - 3):
            if all(board[r+i][c+i] == piece for i in range(4)):
                return True
    # Negative diagonal
    for c in range(COLUMNS - 3):
        for r in range(3, ROWS):
            if all(board[r-i][c+i] == piece for i in range(4)):
                return True
    return False

def is_draw(board):
    return all(board[ROWS-1][c] != 0 for c in range(COLUMNS))


In [4]:
def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

def score_position(board, piece):
    score = 0

    # Center column preference
    center_array = [board[r][COLUMNS//2] for r in range(ROWS)]
    score += center_array.count(piece) * 3

    # Horizontal score
    for r in range(ROWS):
        row_array = board[r]
        for c in range(COLUMNS - 3):
            window = row_array[c:c+4]
            score += evaluate_window(window, piece)

    # Vertical score
    for c in range(COLUMNS):
        col_array = [board[r][c] for r in range(ROWS)]
        for r in range(ROWS - 3):
            window = col_array[r:r+4]
            score += evaluate_window(window, piece)

    # Positive diagonal
    for r in range(ROWS - 3):
        for c in range(COLUMNS - 3):
            window = [board[r+i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    # Negative diagonal
    for r in range(3, ROWS):
        for c in range(COLUMNS - 3):
            window = [board[r-i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score


In [5]:
import copy

def get_valid_locations(board):
    return [c for c in range(COLUMNS) if is_valid_location(board, c)]

def is_terminal_node(board):
    return winning_move(board, PLAYER_PIECE) or winning_move(board, AI_PIECE) or is_draw(board)

def minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    terminal = is_terminal_node(board)

    if depth == 0 or terminal:
        if terminal:
            if winning_move(board, AI_PIECE):
                return (None, 100000000000000)
            elif winning_move(board, PLAYER_PIECE):
                return (None, -10000000000000)
            else:
                return (None, 0)
        else:
            return (None, score_position(board, AI_PIECE))

    if maximizingPlayer:
        value = -math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            b_copy = copy.deepcopy(board)
            row = get_next_open_row(b_copy, col)
            drop_piece(b_copy, row, col, AI_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                best_col = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return best_col, value
    else:
        value = math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            b_copy = copy.deepcopy(board)
            row = get_next_open_row(b_copy, col)
            drop_piece(b_copy, row, col, PLAYER_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                best_col = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return best_col, value


In [6]:
def play_game():
    board = create_board()
    game_over = False
    turn = 0  # 0 for player, 1 for AI
    print_board(board)

    while not game_over:
        if turn == 0:
            col = int(input("Player 1 Make your Selection (1-7): ")) - 1
            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, PLAYER_PIECE)

                if winning_move(board, PLAYER_PIECE):
                    print_board(board)
                    print("PLAYER 1 WINS!")
                    game_over = True
        else:
            col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)
            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, AI_PIECE)
                print(f"AI drops in column {col+1}")

                if winning_move(board, AI_PIECE):
                    print_board(board)
                    print("AI WINS!")
                    game_over = True

        print_board(board)

        if is_draw(board):
            print("DRAW!")
            break

        turn = (turn + 1) % 2


In [7]:
play_game()


0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 2 3 4 5 6 7

Player 1 Make your Selection (1-7): 2
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 0 0 0 0 0
1 2 3 4 5 6 7

AI drops in column 4
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 0 2 0 0 0
1 2 3 4 5 6 7

Player 1 Make your Selection (1-7): 5
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 0 2 1 0 0
1 2 3 4 5 6 7

AI drops in column 4
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 2 0 0 0
0 1 0 2 1 0 0
1 2 3 4 5 6 7

Player 1 Make your Selection (1-7): 4
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 0 0 2 0 0 0
0 1 0 2 1 0 0
1 2 3 4 5 6 7

AI drops in column 2
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 2 0 2 0 0 0
0 1 0 2 1 0 0
1 2 3 4 5 6 7

Player 1 Make your Selection (1-7): 3
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 2 0 2 0 0 0
0 1 1 2 1 0 0
1 2 3 4 

**Question 3**

In [8]:
from collections import defaultdict, deque
import itertools

# ------------------- Problem Data -------------------
packages = [1, 2, 3, 4, 5]
priorities = {
    1: "high", 2: "high", 3: "medium", 4: "medium", 5: "low"
}

domains = {
    1: ['W1->A', 'W1->B'],
    2: ['W1->C', 'W1->D'],
    3: ['A', 'D'],
    4: ['W2->B'],
    5: ['W1->C', 'W2->C']
}

storage_limits = {
    'A': 2, 'B': 3, 'C': 1, 'D': 2,
    'W1': 3, 'W2': 4
}

costs = {
    'W1->A': 3,
    'W1->B': 2,
    'W2->C': 4,
    'A->D': 5,
    'A': 0,
    'B': 0,
    'C': 0,
    'D': 0
}

# ------------------- CSP Class -------------------
class CSP:
    def __init__(self, variables, domains):
        self.variables = variables
        self.domains = domains.copy()
        self.neighbors = self._build_neighbors()
        self.assignment = {}
        self.storage_used = defaultdict(int)

    def _build_neighbors(self):
        neighbors = {var: set() for var in self.variables}
        for i in self.variables:
            for j in self.variables:
                if i != j:
                    neighbors[i].add(j)
        return neighbors

    def is_consistent(self, var, value):
        dest = value.split('->')[-1]
        if self.storage_used[dest] >= storage_limits.get(dest, float('inf')):
            return False
        return True

    def assign(self, var, value):
        self.assignment[var] = value
        dest = value.split('->')[-1]
        self.storage_used[dest] += 1

    def unassign(self, var):
        if var in self.assignment:
            dest = self.assignment[var].split('->')[-1]
            self.storage_used[dest] -= 1
            del self.assignment[var]

    def ac3(self):
        queue = deque([(xi, xj) for xi in self.variables for xj in self.neighbors[xi]])
        while queue:
            xi, xj = queue.popleft()
            if self.revise(xi, xj):
                if not self.domains[xi]:
                    return False
                for xk in self.neighbors[xi] - {xj}:
                    queue.append((xk, xi))
        return True

    def revise(self, xi, xj):
        revised = False
        for x in self.domains[xi][:]:
            if not any(True for y in self.domains[xj]):
                self.domains[xi].remove(x)
                revised = True
        return revised

# ------------------- Heuristics -------------------
def mrv(csp):
    unassigned = [v for v in csp.variables if v not in csp.assignment]
    return min(unassigned, key=lambda var: len(csp.domains[var]))

def lcv(csp, var):
    values = csp.domains[var]
    value_constraints = {}
    for value in values:
        score = 0
        dest = value.split('->')[-1]
        for neighbor in csp.neighbors[var]:
            if value in csp.domains[neighbor]:
                score += 1
        value_constraints[value] = score
    return sorted(values, key=lambda val: value_constraints[val])

# ------------------- Backtracking -------------------
def backtrack(csp):
    if len(csp.assignment) == len(csp.variables):
        return csp.assignment

    var = mrv(csp)
    for value in lcv(csp, var):
        if csp.is_consistent(var, value):
            csp.assign(var, value)
            result = backtrack(csp)
            if result:
                return result
            csp.unassign(var)
    return None

# ------------------- Run the CSP -------------------
csp = CSP(packages, domains)
if csp.ac3():
    result = backtrack(csp)
    if result:
        print("Final Package Assignments:")
        total_cost = 0
        for pkg, route in sorted(result.items()):
            cost = costs.get(route, 0)
            print(f"Package {pkg}: Route = {route}, Cost = {cost}")
            total_cost += cost
        print(f"\nTotal Routing Cost: {total_cost}")
    else:
        print("No solution found.")
else:
    print("AC-3 found inconsistency in constraints.")


Final Package Assignments:
Package 1: Route = W1->A, Cost = 3
Package 2: Route = W1->D, Cost = 0
Package 3: Route = A, Cost = 0
Package 4: Route = W2->B, Cost = 0
Package 5: Route = W2->C, Cost = 4

Total Routing Cost: 7


**Question 4**

In [ ]:
import numpy as np
import random

EMPTY = '.'
PLAYER_X = 'X'
PLAYER_O = 'O'

class SmallBoard:
    def __init__(self):
        self.board = [[EMPTY for _ in range(3)] for _ in range(3)]
        self.winner = None

    def is_full(self):
        return all(cell != EMPTY for row in self.board for cell in row)

    def check_winner(self):
        b = self.board
        for i in range(3):
            if b[i][0] == b[i][1] == b[i][2] != EMPTY:
                self.winner = b[i][0]
                return
            if b[0][i] == b[1][i] == b[2][i] != EMPTY:
                self.winner = b[0][i]
                return
        if b[0][0] == b[1][1] == b[2][2] != EMPTY:
            self.winner = b[0][0]
        elif b[0][2] == b[1][1] == b[2][0] != EMPTY:
            self.winner = b[0][2]

    def make_move(self, row, col, player):
        if self.board[row][col] == EMPTY and not self.winner:
            self.board[row][col] = player
            self.check_winner()
            return True
        return False

    def __str__(self):
        return '\n'.join([' '.join(row) for row in self.board])

class UltimateTicTacToe:
    def __init__(self):
        self.boards = [[SmallBoard() for _ in range(3)] for _ in range(3)]
        self.global_board = [[EMPTY for _ in range(3)] for _ in range(3)]
        self.current_player = PLAYER_X
        self.last_move = None

    def display(self):
        print("\nUltimate Tic-Tac-Toe Board:")
        for i in range(3):
            for row in range(3):
                print(" | ".join(" ".join(self.boards[i][j].board[row]) for j in range(3)))
            print("-" * 29)

    def make_move(self, big_row, big_col, row, col):
        board = self.boards[big_row][big_col]
        success = board.make_move(row, col, self.current_player)
        if success:
            board.check_winner()
            if board.winner:
                self.global_board[big_row][big_col] = self.current_player
            self.last_move = (row, col)
            self.current_player = PLAYER_O if self.current_player == PLAYER_X else PLAYER_X
        return success

    def is_game_over(self):
        g = self.global_board
        for i in range(3):
            if g[i][0] == g[i][1] == g[i][2] != EMPTY:
                return g[i][0]
            if g[0][i] == g[1][i] == g[2][i] != EMPTY:
                return g[0][i]
        if g[0][0] == g[1][1] == g[2][2] != EMPTY:
            return g[0][0]
        if g[0][2] == g[1][1] == g[2][0] != EMPTY:
            return g[0][2]
        if all(cell != EMPTY for row in g for cell in row):
            return 'Draw'
        return None

    def valid_boards(self):
        if not self.last_move:
            return [(i, j) for i in range(3) for j in range(3)]
        r, c = self.last_move
        if self.boards[r][c].winner or self.boards[r][c].is_full():
            return [(i, j) for i in range(3) for j in range(3) if not self.boards[i][j].winner and not self.boards[i][j].is_full()]
        return [(r, c)]

    def valid_moves(self):
        valid = []
        for big_r, big_c in self.valid_boards():
            for i in range(3):
                for j in range(3):
                    if self.boards[big_r][big_c].board[i][j] == EMPTY:
                        valid.append((big_r, big_c, i, j))
        return valid

    def clone(self):
        new_game = UltimateTicTacToe()
        for i in range(3):
            for j in range(3):
                new_game.boards[i][j].board = [row[:] for row in self.boards[i][j].board]
                new_game.boards[i][j].winner = self.boards[i][j].winner
                new_game.global_board[i][j] = self.global_board[i][j]
        new_game.current_player = self.current_player
        new_game.last_move = self.last_move
        return new_game

    def heuristic(self, player):
        score = 0
        for i in range(3):
            for j in range(3):
                if self.global_board[i][j] == player:
                    score += 10
                elif self.global_board[i][j] != EMPTY:
                    score -= 10
        return score

def minimax(game, depth, alpha, beta, maximizing, player):
    result = game.is_game_over()
    if result == player:
        return 100
    elif result == ('O' if player == 'X' else 'X'):
        return -100
    elif result == 'Draw' or depth == 0:
        return game.heuristic(player)

    if maximizing:
        max_eval = -float('inf')
        for move in game.valid_moves():
            clone = game.clone()
            clone.make_move(*move)
            eval = minimax(clone, depth - 1, alpha, beta, False, player)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in game.valid_moves():
            clone = game.clone()
            clone.make_move(*move)
            eval = minimax(clone, depth - 1, alpha, beta, True, player)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def best_move(game, depth, player):
    best_val = -float('inf')
    move_to_make = None
    for move in game.valid_moves():
        clone = game.clone()
        clone.make_move(*move)
        val = minimax(clone, depth - 1, -float('inf'), float('inf'), False, player)
        if val > best_val:
            best_val = val
            move_to_make = move
    return move_to_make

def play_game():
    game = UltimateTicTacToe()
    depth = 3
    human = input("Choose your symbol (X/O): ").upper()
    ai = PLAYER_O if human == PLAYER_X else PLAYER_X

    while not game.is_game_over():
        game.display()
        if game.current_player == human:
            print("Your move.")
            valid = game.valid_moves()
            while True:
                try:
                    big_r = int(input("Big Row (0-2): "))
                    big_c = int(input("Big Col (0-2): "))
                    r = int(input("Small Row (0-2): "))
                    c = int(input("Small Col (0-2): "))
                    if (big_r, big_c, r, c) in valid:
                        game.make_move(big_r, big_c, r, c)
                        break
                    else:
                        print("Invalid move. Try again.")
                except:
                    print("Invalid input. Enter numbers between 0-2.")
        else:
            print("AI is thinking...")
            move = best_move(game, depth, ai)
            game.make_move(*move)

    game.display()
    result = game.is_game_over()
    if result == 'Draw':
        print("Game is a draw!")
    else:
        print(f"{result} wins the game!")

play_game()


Choose your symbol (X/O): X

Ultimate Tic-Tac-Toe Board:
. . . | . . . | . . .
. . . | . . . | . . .
. . . | . . . | . . .
-----------------------------
. . . | . . . | . . .
. . . | . . . | . . .
. . . | . . . | . . .
-----------------------------
. . . | . . . | . . .
. . . | . . . | . . .
. . . | . . . | . . .
-----------------------------
Your move.
Big Row (0-2): 1
Big Col (0-2): 1
Small Row (0-2): 1
Small Col (0-2): 2

Ultimate Tic-Tac-Toe Board:
. . . | . . . | . . .
. . . | . . . | . . .
. . . | . . . | . . .
-----------------------------
. . . | . . . | . . .
. . . | . . X | . . .
. . . | . . . | . . .
-----------------------------
. . . | . . . | . . .
. . . | . . . | . . .
. . . | . . . | . . .
-----------------------------
AI is thinking...

Ultimate Tic-Tac-Toe Board:
. . . | . . . | . . .
. . . | . . . | . . .
. . . | . . . | . . .
-----------------------------
. . . | . . . | O . .
. . . | . . X | . . .
. . . | . . . | . . .
-----------------------------
. . . | . . . | 

**Question 5**

In [2]:
from collections import defaultdict
import copy
from tabulate import tabulate

# Input Data
total_resources = {
    "D": 20,  # Doctors
    "B": 15,  # ICU Beds
    "V": 10,  # Ventilators
    "M": 50   # Medicines
}

patients = {
    "E": 12,  # Emergency Ward
    "C": 15,  # COVID-19 Ward
    "I": 10,  # ICU
    "G": 8,   # General Ward
    "P": 5    # Pediatrics
}

departments = ["E", "C", "I", "G", "P"]
resources = ["D", "B", "V", "M"]

# Domains: Possible values for each resource in each department
def create_domains():
    domains = {}
    for dep in departments:
        domains[dep] = {}
        for res in resources:
            if res == "D":
                domains[dep][res] = list(range(0, total_resources["D"] + 1))
            elif res == "B":
                domains[dep][res] = list(range(0, total_resources["B"] + 1))
            elif res == "V":
                # Pediatrics can't receive ventilators
                if dep == "P":
                    domains[dep][res] = [0]
                else:
                    domains[dep][res] = list(range(0, total_resources["V"] + 1))
            elif res == "M":
                domains[dep][res] = list(range(0, total_resources["M"] + 1))
    return domains

def is_valid(assignment):
    total = defaultdict(int)

    for dep in assignment:
        for res in assignment[dep]:
            total[res] += assignment[dep][res]

    for res in resources:
        if total[res] > total_resources[res]:
            return False

    # ICU must get at least 3 ventilators before others
    if any(dep in assignment and assignment[dep]["V"] > 0 for dep in departments if dep != "I"):
        if "I" not in assignment or assignment["I"]["V"] < 3:
            return False

    # Emergency must get at least 4 doctors before General gets any
    if "G" in assignment and assignment["G"]["D"] > 0:
        if "E" not in assignment or assignment["E"]["D"] < 4:
            return False

    # P must get 0 ventilators
    if "P" in assignment and assignment["P"]["V"] > 0:
        return False

    # COVID-19 patients > 10 → COVID must get at least 5 ICU beds
    if patients["C"] > 10 and "C" in assignment and assignment["C"]["B"] < 5:
        return False

    # ICU beds imply doctors
    for dep in assignment:
        if assignment[dep]["B"] > 0 and assignment[dep]["D"] == 0:
            return False

    # Ventilators imply ICU beds
    for dep in assignment:
        if assignment[dep]["V"] > 0 and assignment[dep]["B"] == 0:
            return False

    # No child patients → Pediatrics gets 0
    if patients["P"] == 0 and "P" in assignment:
        if any(assignment["P"][r] > 0 for r in resources):
            return False

    # ICU: 2 medicines per ICU bed
    if "I" in assignment and assignment["I"]["M"] < 2 * assignment["I"]["B"]:
        return False

    return True

# Forward Checking
def forward_check(domains, assignment, dep, res, val):
    updated_domains = copy.deepcopy(domains)
    updated_domains[dep][res] = [val]

    # Remove values from other departments that violate constraints
    for d in departments:
        if d == dep:
            continue
        for r in resources:
            updated_domains[d][r] = [v for v in updated_domains[d][r] if v + val <= total_resources[r]]
            if not updated_domains[d][r]:  # Domain wipeout
                return None
    return updated_domains

# MRV Heuristic
def select_unassigned_variable(assignment, domains):
    min_remaining = float('inf')
    chosen_dep = None
    for dep in departments:
        if dep not in assignment:
            count = sum(len(domains[dep][r]) for r in resources)
            if count < min_remaining:
                min_remaining = count
                chosen_dep = dep
    return chosen_dep

# Backtracking Search
def backtrack(assignment, domains):
    if len(assignment) == len(departments):
        if is_valid(assignment):
            return assignment
        return None

    dep = select_unassigned_variable(assignment, domains)
    for d in domains[dep]["D"]:
        for b in domains[dep]["B"]:
            for v in domains[dep]["V"]:
                for m in domains[dep]["M"]:
                    assignment[dep] = {"D": d, "B": b, "V": v, "M": m}
                    if is_valid(assignment):
                        new_domains = forward_check(domains, assignment, dep, "D", d)
                        if new_domains:
                            result = backtrack(assignment, new_domains)
                            if result:
                                return result
                    del assignment[dep]
    return None

# Visualize result
def print_allocation_table(assignment):
    headers = ["Department", "Doctors", "ICU Beds", "Ventilators", "Medicines"]
    table = []
    for dep in departments:
        row = [dep, assignment[dep]["D"], assignment[dep]["B"], assignment[dep]["V"], assignment[dep]["M"]]
        table.append(row)
    print(tabulate(table, headers=headers, tablefmt="fancy_grid"))

# Main Execution
domains = create_domains()
assignment = {}
result = backtrack(assignment, domains)

if result:
    print("\n Final Resource Allocation:")
    print_allocation_table(result)
else:
    print(" No valid allocation found.")



 Final Resource Allocation:
╒══════════════╤═══════════╤════════════╤═══════════════╤═════════════╕
│ Department   │   Doctors │   ICU Beds │   Ventilators │   Medicines │
╞══════════════╪═══════════╪════════════╪═══════════════╪═════════════╡
│ E            │         0 │          0 │             0 │           0 │
├──────────────┼───────────┼────────────┼───────────────┼─────────────┤
│ C            │         1 │          5 │             0 │           0 │
├──────────────┼───────────┼────────────┼───────────────┼─────────────┤
│ I            │         0 │          0 │             0 │           0 │
├──────────────┼───────────┼────────────┼───────────────┼─────────────┤
│ G            │         0 │          0 │             0 │           0 │
├──────────────┼───────────┼────────────┼───────────────┼─────────────┤
│ P            │         0 │          0 │             0 │           0 │
╘══════════════╧═══════════╧════════════╧═══════════════╧═════════════╛
